In [1]:
import os
%pwd

'c:\\Resume Projects\\Kidney-Tumor-Classifier\\research'

In [2]:
os.chdir("C:\Resume Projects\Kidney-Tumor-Classifier")
%pwd

'C:\\Resume Projects\\Kidney-Tumor-Classifier'

# Dagshub

In [3]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/sheetalkumari2425/Kidney-Tumor-Classifier.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "sheetalkumari2425"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "7819a81c0e71ef64b5d2f24410aaf2e836a6606c"

In [4]:
import tensorflow as tf

ImportError: Traceback (most recent call last):
  File "c:\Resume Projects\Kidney-Tumor-Classifier\kidneyTumorClassifer\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 62, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: Module use of python39.dll conflicts with this version of Python.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
model = tf.keras.models.load_model("C:\Resume Projects\Kidney-Tumor-Classifier\\artifacts\\training\\model.h5")

: 

# Entity

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

: 

# Configuration

In [ ]:
from KidneyTumorClassifier.constants import *
from KidneyTumorClassifier.utils.common import read_yaml, create_directories, save_json

: 

In [5]:
class ConfigurationManager:

    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(path_of_model="C:\Resume Projects\Kidney-Tumor-Classifier\\artifacts\\training\\model.h5",
                                       training_data="C:\Resume Projects\Kidney-Tumor-Classifier\\artifacts\\data_ingestion\\kidney-ct-scan-image",
                                       mlflow_uri="https://dagshub.com/sheetalkumari2425/Kidney-Tumor-Classifier.mlflow",
                                       all_params=self.params,
                                       params_image_size=self.params.IMAGE_SIZE,
                                       params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

NameError: name 'CONFIG_FILE_PATH' is not defined

# Components

In [6]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

ImportError: Traceback (most recent call last):
  File "c:\Resume Projects\Kidney-Tumor-Classifier\kidneyTumorClassifer\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 62, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: Module use of python39.dll conflicts with this version of Python.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [10]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):
        datagenerator_kwargs = dict(rescale = 1./255, validation_split=0.30)
        dataflow_kwargs = dict(target_size=self.config.params_image_size[:-1],batch_size=self.config.params_batch_size,interpolation="bilinear")
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(directory=self.config.training_data,subset="validation",shuffle=False,
                                                                       **dataflow_kwargs)
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

# Pipeline

In [11]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-09-09 21:36:13,940: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-09 21:36:13,949: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-09 21:36:13,952: INFO: common: Created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 32s 4s/step - loss: 0.1558 - accuracy: 0.9065
[2024-09-09 21:36:46,349: INFO: common: Json file saved as: scores.json]


2024/09/09 21:36:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-09-09 21:36:51,582: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmp6uav899x\model\data\model\assets
[2024-09-09 21:36:53,255: INFO: builder_impl: Assets written to: C:\Users\HP\AppData\Local\Temp\tmp6uav899x\model\data\model\assets]


c:\Resume Projects\Kidney-Tumor-Classifier\kidneyTumorClassifer\lib\site-packages\_distutils_hack\__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2024/09/09 21:38:10 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
